## Секция Импорта библиотек



In [ ]:
import numpy as np                  
import matplotlib.pyplot as plt      
import random                        

from keras.datasets import mnist     # MNIST dataset 
from tensorflow.keras.models import Sequential  # Model Нейронной сети

from tensorflow.keras.layers import Dense, Dropout, Activation # Функции Активации и Слои нейронной сети
from keras.utils import np_utils                      

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.applications import InceptionV3

### Обучить перодвинутую архитектуру с 0 будет очень трудно, так как она имеет большое кол-во параметров.

Давайте попробуем использовать уже обученные нейронные сети для классификации произвольных картинок

In [ ]:
# Импорт бибилиотек
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception 
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import urllib
import cv2
import matplotlib.pyplot as plt
import imageio as io


def predict_image(model_name, image_source):
  
  # Будем использовать этот словарь для выбора модели
  MODELS = {
    "vgg16": VGG16,
    "vgg19": VGG19,
    "inception": InceptionV3,
    "xception": Xception, 
    "resnet": ResNet50
  }
# Определяем входной размер
  input_shape = (224, 224)

  preprocess = imagenet_utils.preprocess_input
  if model_name in ("inception", "xception"):
    input_shape = (299, 299)
    preprocess = preprocess_input

  print("[INFO] loading {}...".format(model_name))
  
  Network = MODELS[model_name]
  model = Network(weights="imagenet") #Мы будем использовать уже обученную нейронную сеть на наборе ImageNet, чтобы сделать выполнить предсказания
  
  print("[INFO] loading and pre-processing image...")
  if type(image_source) == str:
    image = load_img(image_source, target_size=input_shape)
    image = np.resize(image, (input_shape[0], input_shape[1], 3))
    image = img_to_array(image)
  else:
    image = np.resize(image_source, (input_shape[0], input_shape[1], 3))
    image = img_to_array(image)

  image = np.expand_dims(image, axis=0)
  image = preprocess(image)

  print("[INFO] classifying image with '{}'...".format(model_name))
  preds = model.predict(image)
  P = imagenet_utils.decode_predictions(preds)
# ТОП 5 предсказаний
  for (i, (imagenetID, label, prob)) in enumerate(P[0]):
    print("{}. {}: {:.2f}%".format(i + 1, label, prob * 100))
  img = io.imread(image_source)
  (imagenetID, label, prob) = P[0][0]
  cv2.putText(img, "Label: {}, {:.2f}%".format(label, prob * 100), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
  plt.imshow(img)
  plt.axis('off')
  
  return model

In [ ]:
!wget https://image.ibb.co/cuw6pd/soccer_ball.jpg
!wget https://image.ibb.co/hdoVFJ/bmw.png
!wget https://image.ibb.co/h0B6pd/boat.png
!wget https://image.ibb.co/eCyVFJ/clint_eastwood.jpg

In [ ]:
model = predict_image('vgg19', 'bmw.png')

In [ ]:
model.summary()

# Transfer Learning

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
import tensorflow as tf
from tensorflow.keras.datasets import mnist  
from tensorflow import keras
from tensorflow.keras import layers as L
from tensorflow.keras.models import Sequential
import  numpy as np


def change_size(image):
    img = array_to_img(image, scale=False) #returns PIL Image
    img = img.resize((75, 75)) #resize image
    img = img.convert(mode='RGB') #makes 3 channels
    arr = img_to_array(img) #convert back to array
    return arr.astype(np.float64)

In [ ]:
model = Sequential()

model.add(tf.keras.applications.resnet50.ResNet50(input_shape = (75, 75, 3), 
                                include_top = False, 
                                weights = 'imagenet'))

model.add(L.Flatten()) #2048
model.add(L.Dense(128, activation='relu'))
model.add(L.Dense(10, activation='softmax'))

model.compile(optimizer=keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
for layer in model.layers[0].layers:
    if layer.name == 'conv5_block1_0_conv':
        break
    layer.trainable=False

In [ ]:
model.fit(X_train, Y_train,
          batch_size=32, epochs=10,
          verbose=1)

In [ ]:
model.evaluate(X_test,y_test)

## Загружаем тренировочные данные


In [ ]:
#  60,000 28 x 28 pixel тренировочные изображения и  10,000 28 x 28 pixel тестовые изображения
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)

In [ ]:
X_test = np.array(X_test).reshape(-1, 28, 28, 1)
X_test = [change_size(img) for img in X_test]
X_test = np.array(X_test)
X_test.shape

y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:


X_train = np.array(X_train[:30000]).reshape(-1, 28, 28, 1)
X_train = [change_size(img) for img in X_train]
X_train = np.array(X_train)
X_train.shape

In [ ]:
Y_train = tf.keras.utils.to_categorical(y_train)[:30000] # one-hot encoding

AlexNet Pytorch Модель

In [ ]:
import torch
import torch.nn as nn

class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 1000) -> None:
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x